Muhammad Zacky Asy'ari <br>
https://www.linkedin.com/in/muhammad-zacky-asyari/


Dataset is from:<br>
https://catalog.data.gov/dataset/wind-turbine-gearbox-condition-monitoring-vibration-analysis-benchmarking-datasets <br>
You may change the directory location depends on your computer

Publication: <br>
https://ieeexplore.ieee.org/abstract/document/10935342/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import normal
from numpy.random import uniform
import time
# to measure exec time
from timeit import default_timer as timer

# Generate sample

In [2]:
from scipy.io import loadmat

health_path = 'D:/Documents/School/PhD/UI/PhD Research/Code and Dataset/Public dataset/Wind Turbine US Departement of Energy/Healthy/H5.mat'
mat = loadmat(health_path)
print("Keys for healty data: ", mat.keys()) #to see what key for data was called

damage_path = 'D:/Documents/School/PhD/UI/PhD Research/Code and Dataset/Public dataset/Wind Turbine US Departement of Energy/Damaged/D5.mat'
mat = loadmat(damage_path)

Keys for healty data:  dict_keys(['__header__', '__version__', '__globals__', 'AN3', 'AN4', 'AN5', 'AN6', 'AN7', 'AN8', 'AN9', 'AN10', 'Speed'])


# Generate one by one

In [3]:
from stingray import lightcurve
import numpy as np
from stingray.bispectrum import Bispectrum
import matplotlib.pyplot as plt
from sklearn import preprocessing as pre
import cv2
import pandas as pd
from numpy.random import normal
from numpy.random import uniform
# to measure exec time
from timeit import default_timer as timer   
%matplotlib inline 


def mat_to_dataframe(data_path='Damaged/D1.mat',col=None, data_start=None, data_end=None):
    '''This code to convert matlab .mat data to pandas dataframe 
    data_path: path to file location 
    col      : key dictionary/column name at original file (put keys in list)
    data size: amount of row to be generated on the dataframe'''
    mat = loadmat(data_path)
    values = []
    for i in range(len(col)):
        values.append(mat[col[i]][data_start:data_end].flatten()) 
    d = dict(zip(col, values))
    df = pd.DataFrame.from_dict(d)
    return df

# Generate data frame for healty and damage dataset
keys_healthy = ['AN3', 'AN4','AN5', 'AN6', 'AN7','AN8', 'AN9', 'AN10']
keys_damage = ['AN3', 'AN4','AN5', 'AN6', 'AN7','AN8', 'AN9', 'AN10']

health_path= 'Healthy/H1.mat'
damage_path = 'Damaged/D1.mat'

data_start = None
data_end = None
healthy = mat_to_dataframe(data_path=health_path,col=keys_healthy, data_start=data_start, data_end=data_end)
damage = mat_to_dataframe(data_path=damage_path,col=keys_damage, data_start=data_start, data_end=data_end)

Function to crop image

[165:315, 200:350] equal to 150x150 px <br> [140:340, 180:380] equal to 200x200px

In [ ]:
img = cv2.imread('output_bispectrum/cropped_test/AN8_health0_ori.png')
print(img.shape) # Print image shape
cv2.imshow("original", img)
cropped_image = img[140:340, 180:380] # 
# Display cropped image
#cv2.imshow("cropped", cropped_image)            
# Save the cropped image
crop_result = cv2.imwrite('output_bispectrum/cropped_test/AN8_health0_ori_cropped.png', cropped_image)
print(cropped_image.shape)
cv2.destroyAllWindows()

In [4]:
def cropped_image(path=None):
    img = cv2.imread(path.format(i))
    # print(img.shape) # Print image shape
    #cv2.imshow("original", img)
    cropped_image = img[140:340, 180:380]
    # Display cropped image
    #cv2.imshow("cropped", cropped_image)            
    # Save the cropped image
    crop_result = cv2.imwrite(path.format(i), cropped_image)
    cv2.destroyAllWindows()
    return crop_result



# Sensor selection for health

In [16]:
sensor_name = "AN8"
path_file = "output_bispectrum/Full_dataset/AN8_Health/AN8_health{0}.png" # change sensor name according to your preferences

In [13]:
import time

sample = 2
data_start = 0
data_end = 1000
for i in range(sample):
    healthy = mat_to_dataframe(data_path=health_path,col=keys_healthy, data_start=data_start, data_end=data_end)
    #damage = mat_to_dataframe(data_path=damage_path,col=keys_damage, data_start=data_start, data_end=data_end)
    counts = healthy[sensor_name].to_numpy()
    # Variable that can be used for all variable
    times = np.array(np.arange(0, len(healthy[sensor_name])))
    array_health = healthy[sensor_name].to_numpy().reshape(-1, 1)
    array_health_norm = pre.MinMaxScaler(feature_range=(-1, 1)).fit_transform(array_health)
    lc_health = lightcurve.Lightcurve(times,array_health_norm)
    bs_health = Bispectrum(lc_health)  

    resolution = 100
    cont = plt.contourf(bs_health.freq, bs_health.freq, bs_health.bispec_mag, resolution, cmap=plt.cm.Spectral_r)
    plt.colorbar(cont)
    plt.title("Bispectrum Magnitude sample_{0} to {1}".format(data_start,data_end))
    plt.xlabel("freq 1")
    plt.ylabel("freq 2")
    plt.savefig(path_file.format(i+1))
    plt.close()
    # Add function below if you want to crop directly
    time.sleep(0.1)
    cropped_image(path_file.format(i+1))

    data_start += 1000
    data_end += 1000

print("{0} number of samples had been generated".format(i+1))
print("last sample start time: ", data_start)
print("last sample end time: ", data_end)

2 number of samples had been generated
last sample start time:  803000
last sample end time:  804000


# Sensor selection for damage

In [16]:
sensor_name = "AN9"
path_file = "output_bispectrum/Full_dataset/AN9_Damage/AN9_damage{0}.png" # change sensor name according to your preferences

In [17]:
import time

sample = 100
data_start = 0
data_end = 1000
for i in range(sample):
    #healthy = mat_to_dataframe(data_path=health_path,col=keys_healthy, data_start=data_start, data_end=data_end)
    damage = mat_to_dataframe(data_path=damage_path,col=keys_damage, data_start=data_start, data_end=data_end)
    counts = damage[sensor_name].to_numpy()
    # Variable that can be used for all variable
    times = np.array(np.arange(0, len(damage[sensor_name])))
    array_damage = damage[sensor_name].to_numpy().reshape(-1, 1)
    array_damage_norm = pre.MinMaxScaler(feature_range=(-1, 1)).fit_transform(array_damage)
    lc_damage = lightcurve.Lightcurve(times,array_damage_norm)
    bs_damage = Bispectrum(lc_damage)  

    resolution = 100
    cont = plt.contourf(bs_damage.freq, bs_damage.freq, bs_damage.bispec_mag, resolution, cmap=plt.cm.Spectral_r)
    plt.colorbar(cont)
    plt.title("Bispectrum Magnitude sample_{0} to {1}".format(data_start,data_end))
    plt.xlabel("freq 1")
    plt.ylabel("freq 2")
    plt.savefig(path_file.format(i+1))
    plt.close()
    # Add function below if you want to crop directly
    time.sleep(0.1)
    cropped_image(path_file.format(i+1))

    data_start += 1000
    data_end += 1000

print("{0} number of samples had been generated".format(i+1))
print("last sample start time: ", data_start)
print("last sample end time: ", data_end)

100 number of samples had been generated
last sample start time:  100000
last sample end time:  101000
